In [1]:
import Data
import numpy as np
import os
import random

MyFolder = "data/radek/"
files = os.listdir(MyFolder)

In [2]:
channels = ["ax", "ay", "az", "gx", "gy", "gz"]
def toList(data):
    mylist = []
    for channel in channels:
        mylist[channel] = data[channel]
    return mylist

data = []
for file in files:
    data.append(Data.readFile(MyFolder + file))

data2 = []
for sample in data:
    sample = Data.downsample(sample, 2)
    sample = Data.normailizeData(sample)
    sample = Data.getMotionSection(sample)
    data2.append(sample)

samples = Data.splitByChannel(data2)
shortest, longest = Data.findShortestAndLongest(samples)
print(shortest, longest)
samples = Data.fill(samples, longest)

40 206


In [3]:
def splitTrainTest(samples, indicies):
    test = {"ax":[], "ay":[], "az":[], "gx":[], "gy":[], "gz":[], "class":[]}
    train = {"ax":[], "ay":[], "az":[], "gx":[], "gy":[], "gz":[], "class":[]}
    for i in range(len(samples["ax"])):
        if i in indicies:
            for channel in samples:
                test[channel].append(samples[channel][i])
        else:
            for channel in samples:
                train[channel].append(samples[channel][i])
    return train, test

# Determines which indicies of the data belongs to which class and then gets random
# indicies for each class 
def getRandomSamples(data, split):
    classes = data["class"]
    nSplit = int(split * len(classes) / 2)
    classIs = [[],[]]
    for i in range(len(classes)):
        myclass = classes[i]
        classIs[myclass].append(i)
    class0 = random.sample(classIs[0], nSplit)
    class1 = random.sample(classIs[1], nSplit)
    class0.extend(class1)
    return class0

TrainFolder = "data/train/"
ValFolder = "data/val/"
TestFolder = "data/test/"
# Get a list of indicies corresponding to the train/test split
nSamples = len(samples["ax"])
split = .2
testIs = getRandomSamples(samples, split)
train, test = splitTrainTest(samples, testIs)
valIs = getRandomSamples(train, split)
train, val = splitTrainTest(train, valIs)

nTrain = len(train["class"])
nVal = len(val["class"])
nTest = len(test["class"])
print(nTrain, nVal, nTest)

# Sanity check, make sure that there are no samples shared between train, val and test
for sample in samples["ax"]:
    count = 0
    if sample in train["ax"]:
        count+=1
    if sample in val["ax"]:
        count+=1
    if sample in test["ax"]:
        count+=1
    if count > 1:
        raise Exception("Sampling Error")

198 48 60


In [4]:
if not os.path.exists(TrainFolder):
    os.mkdir(TrainFolder)
if not os.path.exists(ValFolder):
    os.mkdir(ValFolder)
if not os.path.exists(TestFolder):
    os.mkdir(TestFolder)
Data.writeData(train, TrainFolder)
Data.writeData(val, ValFolder)
Data.writeData(test, TestFolder)
    #sample = toList(sample)
x,y = Data.readBackFiles(TrainFolder)
print("Train")
print(x.shape)
print(y.shape)
print("Val")
x,y = Data.readBackFiles(ValFolder)
print(x.shape)
print(y.shape)
print("Test")
x,y = Data.readBackFiles(TestFolder)
print(x.shape)
print(y.shape)

Train
(198, 206, 6)
(198, 1)
Val
(48, 206, 6)
(48, 1)
Test
(60, 206, 6)
(60, 1)
